In [2]:
import pennylane as qml
import torch

import numpy as np

In [18]:
H = qml.Hamiltonian(
    coeffs=[-0.5], 
    observables=[qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2)]
)
H_1 = qml.Hamiltonian(
    coeffs=[-0.5, -0.5, -0.5], 
    observables=[qml.PauliZ(0), qml.PauliZ(1), qml.PauliZ(2)]
)

In [19]:
H

  (-0.5) [Z0 Z1 Z2]


In [20]:
H_1

  (-0.5) [Z0]
+ (-0.5) [Z1]
+ (-0.5) [Z2]


In [7]:
init_par = torch.tensor([torch.pi/2, torch.pi/2, torch.pi/2], dtype=torch.float)

# init_par[0] = torch.tensor(0.57)

init_par

tensor([1.5708, 1.5708, 1.5708])